<a href="https://colab.research.google.com/github/yuliiabosher/Cyber_Resilience_Course/blob/main/CDProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=87b9bff3439c1d8807f0add34ef9a0b47b37ad8bd31d3358c7b560b78ded4601
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [2]:
import googlemaps
import os
import requests
import json
from google.colab import output

In [4]:
google_key = input("Enter the Google maps API key: ")
os.environ['GOOGLE_API_KEY'] = google_key
output.clear()

In [41]:
gmaps = googlemaps.Client(os.environ.get('GOOGLE_API_KEY'))

def get_schools(lat,lon,rad):
  school_list=[]
  try:
    places_result = gmaps.places_nearby(location= f"{lat},{lon}", radius = rad, type = "school")
    for place in places_result['results']:
      school_name= place['name']
      school_location_lat= place['geometry']['location']['lat']
      school_location_lng= place['geometry']['location']['lng']
      school_list.append([school_name, school_location_lat,school_location_lng])
    return school_list
  except googlemaps.exceptions.ApiError as e:
    if e.status=="INVALID_REQUEST":
      print("coordinates are invalid")
  except Exception as e:
    print(f"Error: {e}")

In [6]:
aws_key = input("Enter the AWS API key: ")
os.environ['AWS_API_KEY'] = aws_key
output.clear()

In [39]:
def add_schools(lat,lng,rad):
  url = "https://a595c6ya39.execute-api.eu-west-2.amazonaws.com/default/myPythonProjectFunc"

  school_data = get_schools(lat,lng,rad)

  payload = json.dumps({
    "data": school_data
  })
  headers = {
    'Content-Type': 'application/json',
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.text

In [26]:
def show_schools():
  url = "https://a595c6ya39.execute-api.eu-west-2.amazonaws.com/default/myPythonProjectFunc"
  payload = ""
  headers = {
    'x-api-key': os.environ.get('AWS_API_KEY')
  }

  response = requests.request("GET", url, headers=headers, data=payload)
  return response.text

In [44]:
def select_an_action():
  try:
      action = input('select an option \n 1 for add new schools \n 2 for show all schools ')
      if action == "1":
        latitude = float(input('enter a latitude as a decimal '))
        longitude = float(input('enter a longutude as a decimal '))
        radius = float(input('enter a radius as an integer '))
        newly_added_schools = add_schools(latitude,longitude,radius)
        return newly_added_schools
      elif action == "2":
        all_schools = json.loads(show_schools())
        return all_schools['data']
      else:
        return 'you should have entered 1 or 2'
  except Exception as e:
    return f"There was an error {e}"
prompt = select_an_action()
print(prompt)

select an option 
 1 for add new schools 
 2 for show all schools 0
you should have entered 1 or 2
